In [1]:
import requests
import time
from bs4 import BeautifulSoup
import json
import csv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [2]:
# get the website
def get_html(url):
    try:
        r = requests.get(url, timeout = 30)
        r.raise_for_status()
       # r.endcodding = r.apparent_endconding 
        #r.encoding='utf-8'
        return r.text
    except:
        return " ERROR "
        

In [3]:
#driver = webdriver.Chrome('/Users/liyanan/Desktop/chromedriver') 
def get_html(url):
    browser = webdriver.Safari()

#url = 'https://www.truecar.com/hyundai/elantra/reviews/?returnUrl=%2Fprices-new%2Fhyundai%2Felantra-pricing%2F%3Fzipcode%3D27514'

    browser.get(url)
    browser.implicitly_wait(3)

    next_btn = browser.find_elements_by_xpath('//span[@class="read-more-link-text"]')

    for i in range(0,len(next_btn)):
        next_btn[i].click()
    return browser.page_source




In [4]:
#print(len(next_btn))

In [5]:
#page_source = browser.page_source
def get_nextpage(url):
    page_source = get_html(url)
    soup = BeautifulSoup(page_source,'lxml')
    
    pagelist = soup.find_all('a', attrs = {'class': 'page-link'})
    print(pagelist[1].text)
    return soup,pagelist[1]['href']

In [6]:
# clear content use soup
def get_content(url):
    
    #page_source = get_html(url)
    comments = []

    #soup = BeautifulSoup(page_source,'lxml')
    soup = get_nextpage(url)[0]
    
    #print(soup.find_all("div", "align-items-end wrappable-2 padding-1 justify-content-start categoryRatingsContainer_w3m4lu"))

    
    
    litags = soup.find_all('div', attrs = {'class': 'border-bottom'})
    
    
    for li in litags:
        comment = {}

        try:
           # print('testtesttest3')
            comment['date'] = li.find('p',attrs = {'class': 'font-size-1 font-size-md-2 reviewDate_e5cq67'}).text.strip()
            
           # print(comment['date'])
           # if li.find('span', attrs = {'class': 'd-inline-block'})['itemprop'] == 'author':
        
            comment['author'] = li.find('span', attrs = {'itemprop': 'author'}).text.strip()
           # print(comment['author'])
            comment['vehicle'] = li.find('span', attrs = {'itemprop': 'itemReviewed'}).text.strip()
            #print(comment['vehicle'] )
            
            #overall rate
            textrate = li.find('div', attrs = {'class': 'star-rating d-inline-block d-inline-block'})['data-qa']
            if textrate == 'review-star-rating-5':
                comment['overallrate'] = 5
            elif textrate == 'review-star-rating-4':
                comment['overallrate'] = 4
            elif textrate == 'review-star-rating-3':
                comment['overallrate'] = 3
            elif textrate == 'review-star-rating-2':
                comment['overallrate'] = 2
            elif textrate == 'review-star-rating-1':
                comment['overallrate'] = 1
            else:
                comment['overallrate'] = 0
                
            #Comment text
            textlist = li.find_all('div', attrs = {'data-qa': 'Review-review-part'})
            for texts in textlist:
                if comment.get('text', None) == None:
                    comment['text'] = texts.text.strip()
                else:
                    comment['text'] = comment['text'] + texts.text.strip()
            
           
            #Apect rate
            aspectlist = li.find_all('div', attrs = {'data-qa': 'review-category-rating'})
            categoryrates = {}
            for aspects in aspectlist:
                fullstar = len(aspects.find_all('svg', attrs = {'name': 'star-full'}))

                halfstar = len(aspects.find_all('svg', attrs = {'name': 'star-half'}))
   
                categoryrates[aspects.text] = fullstar + halfstar * 0.5
            
            comment['total_category'] = len(categoryrates)
            comment['categoryrates'] = categoryrates
                 
            #print(comment['aspect'])
            
            comments.append(comment)
        except:
            print("ERROR!!!!!!!!!!!")
            
    
    return (comments)

In [7]:
#def out2file(comments):
 #   with open ('truecar.txt', 'a+') as f:
       # f.write('testesttestprint')
        #print('where are you')
  #      for comment in comments:
           # print(comment)
   #         f.write('Author：{} \t Time：{} \n'.format(
    #            comment['author'], comment['date']))
    
    #print('ONE URL DONE!')

In [8]:
def out2file(comments):
    #with open('text1.json', 'w') as f:
     #   json.dump(comments, f)
    with open('fileName2.csv', 'a+') as f:
        writer = csv.writer(f)
        count = 0
        if f.tell() == 0:
            writer.writerow(["author", "date", 'vehicle',"overall_rate",'text','total_category','category_rates'])
        for comment in comments:
            writer.writerow([str(comment['author']), str(comment['date']),str(comment['vehicle']), comment['overallrate'],str(comment['text']),
                             comment['total_category'],comment['categoryrates']
                             ])
            count += 1
    
        
        


In [36]:
#main 
def main(url):
    url_list = [url]
    url_list.append('https://www.truecar.com' + get_nextpage(url)[1])
    
    #collect all url you'll need 
    #for i in range(0,deep):
    
    print('how many')
    print(url_list)
    #url_list.append(base_url + '&pn=' + str(50 * i))
    print("Get all url! Can start to work now!")
    
    for url in url_list:
        content = get_content(url) 
        out2file(content)
    print('ALL URLS DONE!!!!!!!!!!!!!!!!')

In [34]:
#main 
def main(url):
 

    print("Get all url! Can start to work now!")
    content = get_content(url) 
    out2file(content)
    print('ALL URLS DONE!!!!!!!!!!!!!!!!')

In [39]:
#run
main ('https://www.truecar.com/infiniti/q50/reviews/?returnUrl=%2Fprices-new%2Finfiniti%2Fq50-pricing%2F%3Fzipcode%3D27514')




2
how many
['https://www.truecar.com/infiniti/q50/reviews/?returnUrl=%2Fprices-new%2Finfiniti%2Fq50-pricing%2F%3Fzipcode%3D27514', 'https://www.truecar.com/infiniti/q50/reviews/?page=2&per_page=20&returnUrl=%2Fprices-new%2Finfiniti%2Fq50-pricing%2F%3Fzipcode%3D27514']
Get all url! Can start to work now!
2
ERROR!!!!!!!!!!!
ERROR!!!!!!!!!!!

ERROR!!!!!!!!!!!
ERROR!!!!!!!!!!!
ALL URLS DONE!!!!!!!!!!!!!!!!


<title>2020 Hyundai Elantra Prices, Reviews &amp; Incentives | TrueCar</title>